In [ ]:
import pydicom
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

## STEP 1 : Confirmation of equality between csv file list and real file list

In [ ]:
import pandas as pd
import os

ROOT_PATH = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/"

# List of csv file
ori_DF = pd.read_csv(f'{ROOT_PATH}/'+'train_labels.csv')
idx_list = ori_DF['BraTS21ID'].tolist()

str_idx_list = []
for elem in idx_list:
    str_idx = str(elem).zfill(5)
    str_idx_list.append(str_idx)

# List from directory
real_idx = os.listdir(f'{ROOT_PATH}'+'train/') # ['00000', '00002',...

### result
if sorted(str_idx_list) == sorted(real_idx):
    print("All same, no problem")
else:
    print("Something is wrong.!!!")

# STEP 2 : Making dataframe

In [ ]:
df_list = []
SEQ_MRI = ['FLAIR', 'T1w', 'T1wCE', 'T2w']

for idx in real_idx:
    tmp_list = []

    tmp_list.append(idx)
    for seq_dir in SEQ_MRI:
        fullname_path = f'{ROOT_PATH}/train/{idx}/{seq_dir}'
        name_seq = f'n{seq_dir}'
        nSeq = len(os.listdir(fullname_path))
        f = pydicom.dcmread(f'{fullname_path}/{os.listdir(fullname_path)[0]}')
        size_img = (f.Rows, f.Columns)        

        tmp_list.append(seq_dir)
        tmp_list.append(nSeq)
        tmp_list.append(size_img)
        tmp_list.append(f.ImageOrientationPatient)
    df_list.append(tmp_list)

print(df_list[10])
print(np.shape(df_list))

In [ ]:
for idx, elem in enumerate(df_list):
    df_list[idx][0] = int(elem[0])

In [ ]:
col_name = ['BraTS21ID',
            'seq1', 'num1', 'size1', 'ori1',
            'seq2', 'num2', 'size2', 'ori2',
            'seq3', 'num3', 'size3', 'ori3',
            'seq4', 'num4', 'size4', 'ori4']
exp_df = pd.DataFrame(df_list, columns=col_name)

In [ ]:
result_df = pd.merge(basic_df, exp_df, how='left', on='BraTS21ID')

In [ ]:
print(result_df.head())

In [ ]:
result_df.sum()

# STEP 2 : orientation

In [ ]:
import math

https://dicom.innolitics.com/ciods/rt-dose/image-plane/00200037

orientation = [cos(nxx),cos(nxy),cos(nxz),cos(nyx),cos(nyy),cos(nyz)]

cosine graph (y axis) : 1 -> 0 -> -1

degree(x axis) : 0 degree, 90 degree, 180 degree

radian(x axis) : 0 rad, pi/2 rad, pi rad

round -> 1 = 0~45 degree, 0 = 45 ~ 135 degree, -1 = 135 ~ 180 degree

rad -> 1 = 0 ~ pi/4 rad, 0 = pi/4 ~ 3 * pi/4 rad, -1 = 3 * pi/4 ~ pi

In [ ]:
qtr_1 = math.pi / 4
qtr_3 = 3 * math.pi / 4
pi_val = math.pi

print("    pi/4 = ", qtr_1)
print(" 3* pi/4 = ", qtr_3)
print("    pi   = ", pi_val)

In [ ]:
cos_qtr1 = math.cos(math.pi/4)
cos_qtr3 = math.cos(3*math.pi/4)

print("cos( 45deg)= ", cos_qtr1)
print("cos(135deg)= ", cos_qtr3)

In [ ]:
print("1~", cos_qtr1, " = 1")
print(cos_qtr1, " ~ ", cos_qtr3, " = 0")
print(cos_qtr3, " ~ -1 = 0")

In [ ]:
ori_list = ["ori1", "ori2", "ori3", "ori4"]

In [ ]:
import copy

result_df2 = copy.copy(result_df)

In [ ]:
import math
for ori_idx in ori_list:
    for i in range(len(result_df2[ori_idx])):
        tmp = []
    #     print(result_df2["BraTS21ID"][i], "  ", result_df2["ori1"][i], end='\r')
        tmp = list(result_df2[ori_idx][i])
        result_df2[ori_idx][i] = tmp
        for idx, j in enumerate(tmp):
            tmp_2 = float(j)
            if tmp_2 > math.cos(math.pi/4):
                tmp_2 = 1
            if tmp_2 <= math.cos(math.pi/4) and tmp_2 >= math.cos(3*math.pi/4):
                tmp_2 = 0
            if tmp_2 < math.cos(3*math.pi/4):
                tmp_2 = -1
            else:
                pass
    #             tmp_2 = "unknown"
            tmp[idx] = tmp_2
        result_df2[ori_idx][i] = tmp

In [ ]:
print(result_df2)

In [ ]:
sum = 0
for idx in range(len(result_df2)):
    for ori_idx in ori_list:
        tmp_list = result_df2[ori_idx][idx]
        for elem in tmp_list:
            if elem == 1 or elem == 0 or elem == -1 or elem == -0:
                pass
            else:
                sum += 1

In [ ]:
sum

In [ ]:
print(result_df2)

In [ ]:
result_df2["ori1re"] = result_df2["ori1"]
result_df2["ori2re"] = result_df2["ori2"]
result_df2["ori3re"] = result_df2["ori3"]
result_df2["ori4re"] = result_df2["ori4"]

In [ ]:
print(result_df2)

### Thanks to https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data for analysis of image orientation.

In [ ]:
orire_list = ["ori1re", "ori2re", "ori3re", "ori4re"]

In [ ]:
for idx in range(len(result_df2)):
    for ori_idx in orire_list:
        tmp_list = result_df2[ori_idx][idx]
        coords = [tmp_list[0], tmp_list[1], tmp_list[3], tmp_list[4]]
#         print(coords)
        if coords == [1, 0, 0, 0]:
            result_df2[ori_idx][idx] = "coronal"
        elif coords == [1, 0, 0, 1]:
            result_df2[ori_idx][idx] = "axial"
        elif coords == [0, 1, 0, 0]:
            result_df2[ori_idx][idx] = "sagittal"
        else:
            result_df2[ori_idx][idx] = "unknown"

In [ ]:
for idx in orire_list:
    print(idx, " = ",result_df2[idx].unique())

### Q1. How to deal with the variable image size among index?
### Q2. How to process the images with variable file numbers?
### Q3. How to process the images with several other orientations(coronal, axial, sagittal)?

In [ ]:
result_df.head()

In [ ]:
result_df2.head()

In [ ]:
sorted(result_df['size4'].unique())

# thanks to https://www.kaggle.com/josecarmona/btrc-a-simple-tool-to-crop-3d-images

In [ ]:
import numpy as np 
from glob import glob
import pydicom
import matplotlib.pyplot as plt
import plotly.express as px

def read_dicom_files(cohort, case, SEQ_MRI):
    ROOT_PATH = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
    files_glob = f'{ROOT_PATH}/{cohort}/{case}/{SEQ_MRI}/*.dcm'
    sorted_files = sorted(glob(files_glob),key=lambda f: int(f.split('Image-')[1].split('.')[0]))
    return [pydicom.read_file(f) for f in sorted_files]

In [ ]:
def cropped_images(images):
    min=np.array(np.nonzero(images)).min(axis=1)
    max=np.array(np.nonzero(images)).max(axis=1)
    return images[min[0]:max[0],min[1]:max[1],min[2]:max[2]]

In [ ]:
cohort = 'train'
case = '00000'
mpMRI = 'FLAIR'
dicom_files = read_dicom_files(cohort, case, mpMRI)

images = np.array([s.pixel_array for s in dicom_files])
images = cropped_images(images)

fig = px.imshow(images, animation_frame=0, binary_string=True, labels=dict(animation_frame="scan"), height=600)
fig.show()

In [ ]:
import pydicom
import os
import pandas as pd

IDX_list = os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/')
SEQ_list = ['FLAIR', 'T1w', 'T1wCE', 'T2w']

result = []

sum = 0

for idx_elem in IDX_list:
    for seq_elem in SEQ_list:
        PATH = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{idx_elem}/{seq_elem}/'
        OBJ_list = os.listdir(PATH)
        for obj_elem in OBJ_list:
            obj_img = pydicom.dcmread(PATH+obj_elem)
            position_obj = obj_img.PatientPosition
            elem_result = [idx_elem, seq_elem, obj_elem, position_obj]
            result.append(elem_result)
            if obj_elem != "HFS":
                sum += 1
            else:
                pass

print("Non_HFS : ", sum)

col_name = ['IDX', 'SEQ', 'OBJ', 'POSITION']
DF_POSITION = pd.DataFrame(result, columns=col_name)
DF_POSITION.describe()

In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from sklearn.linear_model import LogisticRegression as LRModel
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
if os.path.exists("../input/rsna-miccai-brain-tumor-radiogenomic-classification"):
    data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
    pytorch3dpath = "../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D"
else:
    data_directory = '/media/roland/data/kaggle/rsna-miccai-brain-tumor-radiogenomic-classification'
    pytorch3dpath = "EfficientNet-PyTorch-3D"
    
mri_types = ['FLAIR','T1w','T1wCE','T2w']
SIZE = 256
NUM_IMAGES = 64

In [ ]:
sys.path.append(pytorch3dpath)

In [ ]:
def load_dicom_image(path, img_size=SIZE, voi_lut=True, rotate=0):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if rotate > 0:
        rot_choices = [0, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
        data = cv2.rotate(data, rot_choices[rotate])
        
    data = cv2.resize(data, (img_size, img_size))
    return data

In [ ]:
def load_dicom_images_3d(scan_id, num_imgs=NUM_IMAGES, img_size=SIZE, mri_type="FLAIR", split="train", rotate=0):

    files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f, rotate=rotate) for f in files[p1:p2]]).T 
    if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)
        
    if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)
            
    return np.expand_dims(img3d,0)

In [ ]:
a = load_dicom_images_3d("00000")
print(a.shape)
print(np.min(a), np.max(a), np.mean(a), np.median(a))

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

In [ ]:
set_seed(12)

In [ ]:
train_df = pd.read_csv(f"{data_directory}/train_labels.csv")
display(train_df)

In [ ]:
df_train, df_valid = sk_model_selection.train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=12, 
    stratify=train_df["MGMT_value"],
)

In [ ]:
df_train.tail()

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, mri_type=None, label_smoothing=0.01, split="train", augment=False):
        self.paths = paths
        self.targets = targets
        self.mri_type = mri_type
        self.label_smoothing = label_smoothing
        self.split = split
        self.augment = augment
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split=self.split)
        else:
            if self.augment:
                rotation = np.random.randint(0,4)
            else:
                rotation = 0

            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split="train", rotate=rotation)

        if self.targets is None:
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            y = torch.tensor(abs(self.targets[index]-self.label_smoothing), dtype=torch.float)
            return {"X": torch.tensor(data).float(), "y": y}

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s            ",
                n_epoch, train_loss, train_time
            )
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            # if True:
            # if self.best_valid_score < valid_auc: 
            if self.best_valid_score > valid_loss: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_loss, self.lastmodel
                )
                self.best_valid_score = valid_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0

        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            
            loss = self.criterion(outputs, targets)
            loss.backward()

            sum_loss += loss.detach().item()

            self.optimizer.step()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)

                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)

                sum_loss += loss.detach().item()
                y_all.extend(batch["y"].tolist())
                outputs_all.extend(outputs.tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        y_all = [1 if x > 0.5 else 0 for x in y_all]
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_mri_type(df_train, df_valid, mri_type):
    if mri_type=="all":
        train_list = []
        valid_list = []
        for mri_type in mri_types:
            df_train.loc[:,"MRI_Type"] = mri_type
            train_list.append(df_train.copy())
            df_valid.loc[:,"MRI_Type"] = mri_type
            valid_list.append(df_valid.copy())

        df_train = pd.concat(train_list)
        df_valid = pd.concat(valid_list)
    else:
        df_train.loc[:,"MRI_Type"] = mri_type
        df_valid.loc[:,"MRI_Type"] = mri_type

    print(df_train.shape, df_valid.shape)
    display(df_train.head())
    
    train_data_retriever = Dataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values,
        augment=True
    )

    valid_data_retriever = Dataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        df_valid["MRI_Type"].values
    )

    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=32,
        shuffle=True,
        num_workers=8,
    )

    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=32,
        shuffle=False,
        num_workers=8,
    )

    model = Model()
    model.to(device)

    #checkpoint = torch.load("best-model-all-auc0.555.pth")
    #model.load_state_dict(checkpoint["model_state_dict"])

    #print(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        10, 
        train_loader, 
        valid_loader, 
        f"{mri_type}", 
        10,
    )
    
    return trainer.lastmodel

In [ ]:
modelfiles = None

In [ ]:
from efficientnet_pytorch_3d import EfficientNet3D

In [ ]:
if not modelfiles:
    modelfiles = [train_mri_type(df_train, df_valid, m) for m in mri_types]
    print(modelfiles)

In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from sklearn.linear_model import LogisticRegression as LRModel
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
if os.path.exists("../input/rsna-miccai-brain-tumor-radiogenomic-classification"):
    ROOT_DIR = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
else:
    ROOT_DIR = '/media/roland/data/kaggle/rsna-miccai-brain-tumor-radiogenomic-classification'

In [ ]:
def load_dicom_image(path, img_size=SIZE, voi_lut=True, rotate=0):
    """
    Function: translate from dicom file to array with windowing,
    rotation, resizing

    Arguments:
    [0] path,
    [1] image_size: single number,
    [2] voi_lut: windowing,
    [3] optional opencv rotate

    Return: 2D array list (numpy?)
    """
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if rotate > 0:
        rot_choices = [0, cv2.ROTATE_90_CLOCKWISE,
        cv2.ROTATE_90_COUNTERCLOCKWISE,
        cv2.ROTATE_180]
        data = cv2.rotate(data, rot_choices[rotate])
        
    data = cv2.resize(data, (img_size, img_size))
    return data

In [ ]:
def load_dicom_images_3d(
    scan_id,
    num_imgs=NUM_IMAGES,
    img_size=SIZE,
    mri_type="FLAIR",
    split="train",
    rotate=0):
    """
    Function: read the dicom image by scan id, return
    Arguments:
    Return:
    """
    files = sorted(glob.glob(f"{ROOT_DIR}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f, rotate=rotate) for f in files[p1:p2]]).T 
    if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)
    if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)
    return np.expand_dims(img3d,0)

In [ ]:
a = load_dicom_images_3d("00000")
print(a.shape)
print(np.min(a), np.max(a), np.mean(a), np.median(a))

In [ ]:
def set_seed(seed):
    """
    set the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

In [ ]:
set_seed(12)

In [ ]:
train_df = pd.read_csv(f"{ROOT_DIR}/train_labels.csv")
print(train_df)

In [ ]:
df_train, df_valid = sk_model_selection.train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=12, 
    stratify=train_df["MGMT_value"],
)

In [ ]:
df_train.tail()

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, mri_type=None, label_smoothing=0.01, split="train", augment=False):
        self.paths = paths
        self.targets = targets
        self.mri_type = mri_type
        self.label_smoothing = label_smoothing
        self.split = split
        self.augment = augment
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split=self.split)
        else:
            if self.augment:
                rotation = np.random.randint(0,4)
            else:
                rotation = 0
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split="train", rotate=rotation)
        if self.targets is None:
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            y = torch.tensor(abs(self.targets[index]-self.label_smoothing), dtype=torch.float)
            return {"X": torch.tensor(data).float(), "y": y}

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        out = self.net(x)
        return out
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x, dim=1)

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s            ",
                n_epoch, train_loss, train_time
            )
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )
            # if True:
            # if self.best_valid_score < valid_auc: 
            if self.best_valid_score > valid_loss: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_loss, self.lastmodel
                )
                self.best_valid_score = valid_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0
        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            loss = self.criterion(outputs, targets)
            loss.backward()
            sum_loss += loss.detach().item()
            self.optimizer.step()
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        return sum_loss/len(train_loader), int(time.time() - t)
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []
        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)
                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)
                sum_loss += loss.detach().item()
                y_all.extend(batch["y"].tolist())
                outputs_all.extend(outputs.tolist())
            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
        y_all = [1 if x > 0.5 else 0 for x in y_all]
        auc = roc_auc_score(y_all, outputs_all)
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_mri_type(df_train, df_valid, mri_type):
    if mri_type=="all":
        train_list = []
        valid_list = []
        for mri_type in mri_types:
            df_train.loc[:,"MRI_Type"] = mri_type
            train_list.append(df_train.copy())
            df_valid.loc[:,"MRI_Type"] = mri_type
            valid_list.append(df_valid.copy())
        df_train = pd.concat(train_list)
        df_valid = pd.concat(valid_list)
    else:
        df_train.loc[:,"MRI_Type"] = mri_type
        df_valid.loc[:,"MRI_Type"] = mri_type
    print(df_train.shape, df_valid.shape)
    print(df_train.head())
    train_data_retriever = Dataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values,
        augment=True
    )
    valid_data_retriever = Dataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        df_valid["MRI_Type"].values
    )
    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=32,
        shuffle=True,
        num_workers=8,
    )
    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=32,
        shuffle=False,
        num_workers=8,
    )
    model = Model()
    model.to(device)
    #checkpoint = torch.load("best-model-all-auc0.555.pth")
    #model.load_state_dict(checkpoint["model_state_dict"])
    #print(model)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = torch_functional.binary_cross_entropy_with_logits
    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )
    history = trainer.fit(
        10, 
        train_loader, 
        valid_loader, 
        f"{mri_type}", 
        10,
    )
    return trainer.lastmodel

In [ ]:
modelfiles = None
# modelfiles = ["/kaggle/input/efficientnettrainedmodels/FLAIR-e10-loss0.680-auc0.624.pth",
#               "/kaggle/input/efficientnettrainedmodels/T1w-e6-loss0.691-auc0.581.pth",
#               "/kaggle/input/efficientnettrainedmodels/T1wCE-e1-loss0.693-auc0.500.pth",
#               "/kaggle/input/efficientnettrainedmodels/T2w-e3-loss0.694-auc0.570.pth"]

if not modelfiles:
    modelfiles = [train_mri_type(df_train, df_valid, m) for m in mri_types]
    print(modelfiles)

In [ ]:
def predict(modelfile, df, mri_type, split):
    print("Predict:", modelfile, mri_type, df.shape)
    df.loc[:,"MRI_Type"] = mri_type
    data_retriever = Dataset(
        df.index.values, 
        mri_type=df["MRI_Type"].values,
        split=split
    )

    data_loader = torch_data.DataLoader(
        data_retriever,
        batch_size=32,
        shuffle=False,
        num_workers=8,
    )
   
    model = Model()
    model.to(device)
    
    checkpoint = torch.load(modelfile)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch["X"].to(device))).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            ids.extend(batch["id"].numpy().tolist())
            
    preddf = pd.DataFrame({"BraTS21ID": ids, "MGMT_value": y_pred}) 
    preddf = preddf.set_index("BraTS21ID")
    return preddf

In [ ]:
df_train = df_train.set_index("BraTS21ID")

for m, mtype in zip(modelfiles,  mri_types):
    pred = predict(m, df_train, mtype, "train")
    df_train[mtype] = pred["MGMT_value"]

y = df_train.values[:, 0]
y = [np.int64(i) for i in y]
y = np.array(y)
X = df_train.values[:, 2:6]

emsemble_LR = LRModel(random_state=0).fit(X, y)

df_valid = df_valid.set_index("BraTS21ID")
for m, mtype in zip(modelfiles,  mri_types):
    pred = predict(m, df_valid, mtype, "train")
    df_valid[mtype] = pred["MGMT_value"]
X_valid = df_valid.values[:, 2:6]

y_pred = emsemble_LR.predict_proba(X_valid)
df_valid["MGMT_pred"] = y_pred[:, 1]
auc = roc_auc_score(df_valid["MGMT_value"], df_valid["MGMT_pred"])
print(f"Validation ensemble AUC: {auc:.4f}")
sns.displot(df_valid["MGMT_pred"])

df_valid["Average"] = 0
for mtype in mri_types:
    df_valid["Average"] += df_valid[mtype]
df_valid["Average"] /= len(modelfiles)
auc = roc_auc_score(df_valid["MGMT_value"], df_valid["Average"])
print(f"Validation ensemble AUC: {auc:.4f}")
sns.displot(df_valid["MGMT_pred"])